# OC PROJET 5 - AUTOMATICALLY CATEGORIZE QUESTIONS
#### CLEANING AND ANALYSIS NOTEBOOK
<br></br>
### SOMMAIRE
- <a href="#C1">I. Nettoyage des données</a>
    
- <a href="#C2">II. Feature Engineering</a>

- <a href="#C3">III. Preprocessing</a>
    
- <a href="#C4">IV. Exploration des données</a>
    - 1. Matrice des corrélations
    - 2. Analyse temporelle
    - 3. Analyse Quanti/Quanti
    - 4. Analyse Quanti/Quali
    - 5. Analyse Quali/Quali
    - 6. ACP

# <a name="C1">I. Nettoyage et fusion des données</a>

<font size="5">1. Importation des librairies</font>

In [1]:
# importation des librairies
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import font_manager as rcParams
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
from scipy.stats import pearsonr
from scipy.stats import f_oneway
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
import nltk
import enchant
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize

<font size="5">2. Paramétrages Data Visualisation</font>

In [2]:
# Ajouter une ombre à la police
shadow = path_effects.withSimplePatchShadow(offset = (1, - 0.75), 
shadow_rgbFace = 'darkblue', alpha = 0.25)

# changer la police dans les graphiques, les couleurs 
# et augmenter la résolution d'affichage
plt.rcParams['font.family'] = 'Ebrima'
plt.rcParams['text.color'] = 'white'
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200
plt.style.use('dark_background')

# set le theme seaborn
sns.set_style('darkgrid', {'axes.facecolor': '0.2',
'text.color': 'white', 'figure.figsize': (20, 16)})
plt.rcParams['figure.facecolor'] = '0.2'

# suppression de l'affichage max des colonnes
pd.set_option('display.max_columns', None)

<font size="5">3. Requêtes SQL</font>

SELECT TOP 50000 Title, Body, Tags, Id, Score, ViewCount, AnswerCount, CreationDate, LastActivityDate, CommentCount<font size="5">2. Paramétrages Data Visualisation</font>
FROM Posts
WHERE PostTypeId = 1 
  AND ViewCount > 100 
  AND Score > 3 
  AND AnswerCount > 0 
  AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5 
  AND CommentCount > 0
ORDER BY Id ASC

SELECT TOP 50000 Title, Body, Tags, Id, Score, ViewCount, AnswerCount, CreationDate, LastActivityDate, CommentCount
FROM Posts
WHERE PostTypeId = 1 
  AND ViewCount > 100 
  AND Score > 3 
  AND AnswerCount > 0 
  AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5 
  AND CommentCount > 0
ORDER BY Id DESC

<font size="5">4. Dataframe</font>

In [3]:
df1 = pd.read_csv('QueryResultsAsc.csv')
df2 = pd.read_csv('QueryResultsDesc.csv')

In [4]:
# Fusion des DataFrames df1 et df2 sur la colonne commune
df = pd.concat([df1, df2])
df.shape

(100000, 10)

In [32]:
df.head()

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount
0,How to convert Decimal to Double in C#?,"<p>I want to assign the decimal variable &quot;trans&quot; to the double variable &quot;this.Opacity&quot;.</p>\n<pre class=""lang-cs prettyprint-override""><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n<p>When I build the app it gives the following error:</p>\n<blockquote>\n<p>Cannot implicitly convert type decimal to double</p>\n</blockquote>\n",<c#><floating-point><type-conversion><double><decimal>,4,795,72107,13,2008-07-31 21:42:52,2022-09-08 05:07:26,4
1,Calculate relative time in C#,"<p>Given a specific <code>DateTime</code> value, how do I display relative time, like:</p>\n<ul>\n<li><code>2 hours ago</code></li>\n<li><code>3 days ago</code></li>\n<li><code>a month ago</code></li>\n</ul>\n",<c#><datetime><time><datediff><relative-time-span>,11,1645,198982,41,2008-07-31 23:55:37,2022-09-05 11:26:30,3
2,Determine a user's timezone,<p>Is there a standard way for a web server to be able to determine a user's timezone within a web page? </p>\n\n<p>Perhaps from an HTTP header or part of the <code>user-agent</code> string?</p>\n,<html><browser><timezone><user-agent><timezone-offset>,13,704,285740,27,2008-08-01 00:42:38,2022-03-29 07:31:31,10
3,What is the fastest way to get the value of π?,"<p>I'm looking for the fastest way to obtain the value of π, as a personal challenge. More specifically, I'm using ways that don't involve using <code>#define</code> constants like <code>M_PI</code>, or hard-coding the number in.</p>\n\n<p>The program below tests the various ways I know of. The inline assembly version is, in theory, the fastest option, though clearly not portable. I've included it as a baseline to compare against the other versions. In my tests, with built-ins, the <code>4 * atan(1)</code> version is fastest on GCC 4.2, because it auto-folds the <code>atan(1)</code> into a constant. With <code>-fno-builtin</code> specified, the <code>atan2(0, -1)</code> version is fastest.</p>\n\n<p>Here's the main testing program (<code>pitimes.c</code>):</p>\n\n<pre class=""lang-c prettyprint-override""><code>#include &lt;math.h&gt;\n#include &lt;stdio.h&gt;\n#include &lt;time.h&gt;\n\n#define ITERS 10000000\n#define TESTWITH(x) { \\n diff = 0.0; \\n time1 = clock(); \\n for (i = 0; i &lt; ITERS; ++i) \\n diff += (x) - M_PI; \\n time2 = clock(); \\n printf(""%s\t=&gt; %e, time =&gt; %f\n"", #x, diff, diffclock(time2, time1)); \\n}\n\nstatic inline double\ndiffclock(clock_t time1, clock_t time0)\n{\n return (double) (time1 - time0) / CLOCKS_PER_SEC;\n}\n\nint\nmain()\n{\n int i;\n clock_t time1, time2;\n double diff;\n\n /* Warmup. The atan2 case catches GCC's atan folding (which would\n * optimise the ``4 * atan(1) - M_PI'' to a no-op), if -fno-builtin\n * is not used. */\n TESTWITH(4 * atan(1))\n TESTWITH(4 * atan2(1, 1))\n\n#if defined(__GNUC__) &amp;&amp; (defined(__i386__) || defined(__amd64__))\n extern double fldpi();\n TESTWITH(fldpi())\n#endif\n\n /* Actual tests start here. */\n TESTWITH(atan2(0, -1))\n TESTWITH(acos(-1))\n TESTWITH(2 * asin(1))\n TESTWITH(4 * atan2(1, 1))\n TESTWITH(4 * atan(1))\n\n return 0;\n}\n</code></pre>\n\n<p>And the inline assembly stuff (<code>fldpi.c</code>) that will only work for x86 and x64 systems:</p>\n\n<pre class=""lang-c prettyprint-override""><code>double\nfldpi()\n{\n double pi;\n asm(""fldpi"" : ""=t"" (pi));\n return pi;\n}\n</code></pre>\n\n<p>And a build script that builds all the configurations I'm testing (<code>build.sh</code>):</p>\n\n<pre><code>#!/bin/sh\ngcc -O3 -Wall -c -m32 -o fldpi-32.o fldpi.c\ngcc -O3 -Wall -c -m64 -o fldpi-64.o fldpi.c\n\ngcc -O3 -Wall -ffast-math -m32 -o pitimes1-32 pitimes.c fldpi-32.o\ngcc -O3 -Wall -m32 -o pitimes2-32 pitimes.c fldpi-32.o -lm\ngcc -O3 -Wall -fno-builtin -m32 -o pitimes3-32 pitimes.c fldpi-32.o -lm\ngcc -O3 -Wall -ffast-math -m64 -o pitimes1-64 pitimes.c fldpi-64.o -lm\ng

In [15]:
df.isnull().sum()

Title               0
Body                0
Tags                0
Id                  0
Score               0
ViewCount           0
AnswerCount         0
CreationDate        0
LastActivityDate    0
CommentCount        0
dtype: int64

In [16]:
df.loc[df.duplicated(keep = False),:]

,Title,Body,Tags,Id,Score,ViewCount,AnswerCount,CreationDate,LastActivityDate,CommentCount


In [17]:
df.dtypes

Title               object
Body                object
Tags                object
Id                   int64
Score                int64
ViewCount            int64
AnswerCount          int64
CreationDate        object
LastActivityDate    object
CommentCount         int64
dtype: object

In [18]:
df.describe()

,Id,Score,ViewCount,AnswerCount,CommentCount
count,1.000000e+05,100000.000000,1.000000e+05,100000.000000,100000.00000
mean,3.140196e+07,24.862830,2.314290e+04,3.326830,3.57137
std,2.647337e+07,172.320926,1.223877e+05,3.746832,3.23915
min,4.000000e+00,4.000000,1.010000e+02,1.000000,1.00000
25%,5.628995e+06,5.000000,1.734000e+03,1.000000,1.00000
50%,2.863875e+07,7.000000,4.754000e+03,2.000000,3.00000
75%,5.611157e+07,13.000000,1.338200e+04,4.000000,5.00000
max,7.639181e+07,25651.000000,1.278321e+07,134.000000,51.00000


In [33]:
df['Body'].head()

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [5]:
def preprocess_text(text):
    # Supprimer les balises HTML
    text = re.sub('<.*?>', '', text)
    
    # Convertir en minuscules
    text = text.lower()
    
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatisation des mots
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoindre les tokens prétraités en une seule chaîne de texte
    processed_text = ' '.join(tokens)
    
    return processed_text

In [6]:
df['body_preprocess'] = df['Body'].apply(preprocess_text)

In [7]:
pd.set_option('display.max_colwidth', None)
df['body_preprocess'].head(10)

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# !!!

In [11]:
import platform
import ctypes
import sys

if platform.system() == "Windows":
    kernel32 = ctypes.windll.kernel32
    mem_limit = 8 * 1024 * 1024 * 1024  # 8 Go

    if sys.version_info.major >= 3 and sys.version_info.minor >= 8:
        kernel32.SetProcessWorkingSetSizeEx(ctypes.c_void_p(), ctypes.c_size_t(mem_limit), ctypes.c_size_t(mem_limit), 0)
    else:
        kernel32.SetProcessWorkingSetSize(ctypes.c_void_p(), ctypes.c_size_t(-1), ctypes.c_size_t(mem_limit))
else:
    print("La limitation de mémoire n'est pas supportée sur cette plateforme.")

In [17]:
def remove_nonexistent_words(text):
    words = text.split()
    english_dict = enchant.Dict("en_US")  # Utilisez le dictionnaire correspondant à votre langue

    valid_words = []
    for word in words:
        if english_dict.check(word):
            valid_words.append(word)

    return ' '.join(valid_words)

In [ ]:
df['body_clean'] = df['body_preprocess'].apply(remove_nonexistent_words)